In [11]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [12]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [13]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [14]:
model_tag = 'llama3'  
dataset_tag = '20_newsgroups'
prompt = 'emb'
temp = 0
number_of_experimet = 'dostawka'
selected_samples =  'gaussian'

In [15]:
model = get_model_by_tag(model_tag, token, temp)  
dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)[1264:]
prompt_txt = get_prompt_template(dataset_tag, prompt)
examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)


In [16]:
dataset_for_annotation.columns

Index(['label', 'text'], dtype='object')

In [17]:
print(model, dataset_for_annotation, prompt_txt, examples_for_prompt, output_path)

Llama3_1LLM
Params: {}                        label  \
1264  soc.religion.christian   
1265             alt.atheism   
1266           comp.graphics   
1267               sci.crypt   
1268      rec.sport.baseball   
...                      ...   
1995               sci.crypt   
1996         sci.electronics   
1997         rec.motorcycles   
1998               sci.space   
1999               sci.crypt   

                                                   text  
1264  course id still recommend michael read true re...  
1265  article gregg jaeger writes hed precise reject...  
1266  anyone know software allow convert coreldraw c...  
1267  writes look collectively power throw bums dont...  
1268  anyone idea get japanese league stats regularl...  
...                                                 ...  
1995  david sternlight writes article karl barrus wr...  
1996  hello im looking information alphanumeric page...  
1997  john r daker writes would like offocially nomi...  
1998  articl

In [18]:
text_col = "text"
label_col = "label"
output_col = "output"

In [19]:
annotator = LLMAnnotator(
    model=model,
    dataset=dataset_for_annotation,
    examples_for_prompt=examples_for_prompt,
    prompt_template=prompt_txt,
    column_text=text_col,
    column_label=label_col,
    column_output=output_col
)


In [20]:
df = annotator.get_results(is_save=True, output_path=output_path)

Nr: 0 Predicted label: talk.religion.misc
Nr: 1 Predicted label: talk.religion.misc
Nr: 2 Predicted label: comp.graphics
Nr: 3 Predicted label: talk.politics.misc
Nr: 4 Predicted label: rec.sport.baseball
Nr: 5 Predicted label: talk.politics.guns
Nr: 6 Predicted label: rec.motorcycles
Nr: 7 Predicted label: sci.electronics
Nr: 8 Predicted label: comp.windows.x
Nr: 9 Predicted label: talk.politics.mideast
Nr: 10 Predicted label: sci.electronics
Nr: 11 Predicted label: comp.sys.ibm.pc.hardware
Nr: 12 Predicted label: rec.sport.hockey
Nr: 13 Predicted label: rec.motorcycles
Nr: 14 Predicted label: rec.motorcycles
Nr: 15 Predicted label: talk.politics.misc
Nr: 16 Predicted label: comp.sys.mac.hardware
Nr: 17 Predicted label: sci.space
Nr: 18 Predicted label: sci.space
Nr: 19 Predicted label: soc.religion.christian
Nr: 20 Predicted label: misc.forsale
Nr: 21 Predicted label: comp.graphics
Nr: 22 Predicted label: talk.politics.misc
Nr: 23 Predicted label: talk.religion.misc
Nr: 24 Predicted 

In [21]:
annotator.save_results(output_path=output_path)

Results saved successfully to ./results/emb_temp0/llama3/20_newsgroups_llama3_emb_gaussian_temp0_expdostawka.csv


In [22]:
df = pd.read_csv(output_path)
df

,text,output,logprobs,top_logprobs,original_label
0,course id still recommend michael read true re...,talk.religion.misc,"{'content': [{'token': 'talk', 'logprob': -0.9...","[{'token': 'talk', 'logprob': -0.9835627079010...",soc.religion.christian
1,article gregg jaeger writes hed precise reject...,talk.religion.misc,"{'content': [{'token': 'talk', 'logprob': -0.4...","[{'token': 'talk', 'logprob': -0.4394701719284...",alt.atheism
2,anyone know software allow convert coreldraw c...,comp.graphics,"{'content': [{'token': 'comp', 'logprob': -0.0...","[{'token': 'comp', 'logprob': -0.0184168349951...",comp.graphics
3,writes look collectively power throw bums dont...,talk.politics.misc,"{'content': [{'token': 'talk', 'logprob': -0.0...","[{'token': 'talk', 'logprob': -0.0023419829085...",sci.crypt
4,anyone idea get japanese league stats regularl...,rec.sport.baseball,"{'content': [{'token': 'rec', 'logprob': -0.00...","[{'token': 'rec', 'logprob': -0.00597215956076...",rec.sport.baseball
...,...,...,...,...,...
731,david sternlight writes article karl barrus wr...,sci.crypt,"{'content': [{'token': 'sci', 'logprob': -0.17...","[{'token': 'sci', 'logprob': -0.17696242034435...",sci.crypt
732,hello im looking information alphanumeric page...,sci.electronics,"{'content': [{'token': 'sci', 'logprob': -0.02...","[{'token': 'sci', 'logprob': -0.02991333045065...",sci.electronics
733,john r daker writes would like offocially nomi...,rec.autos,"{'content': [{'token': 'rec', 'logprob': -0.08...","[{'token': 'rec', 'logprob': -0.08121661096811...",rec.motorcycles
734,article writes looking information concerning ...,sci.space,"{'content': [{'token': 'sci', 'logprob': -0.00...","[{'token': 'sci', 'logprob': -0.00035386974923...",sci.space


In [23]:
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    """
    Wyodrębnia pierwszy wyraz objęty nawiasami [] z kolumny `read_col`
    i zapisuje go do kolumny `write_col`.
    
    :param df: DataFrame wejściowy
    :param read_col: Nazwa kolumny z której odczytywany jest tekst
    :param write_col: Nazwa kolumny do której zapisywany będzie wynik
    :return: DataFrame z nową kolumną
    """
    def extract_first_bracketed(text):
        match = re.search(r'\[([^\[\]]+?)\]', str(text))
        return match.group(1) if match else None
    
    df[write_col] = df[read_col].apply(extract_first_bracketed)
    return df


In [24]:
import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df = extract_bracketed_word(df, output_col, 'extracted_label')
    df.to_csv(path, index=False)
    
    return df


In [25]:
df_new = validate_text_column(output_path, examples_for_prompt)
df_new

,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples,extracted_label
0,course id still recommend michael read true re...,talk.religion.misc,"{'content': [{'token': 'talk', 'logprob': -0.9...","[{'token': 'talk', 'logprob': -0.9835627079010...",soc.religion.christian,False,None
1,article gregg jaeger writes hed precise reject...,talk.religion.misc,"{'content': [{'token': 'talk', 'logprob': -0.4...","[{'token': 'talk', 'logprob': -0.4394701719284...",alt.atheism,False,None
2,anyone know software allow convert coreldraw c...,comp.graphics,"{'content': [{'token': 'comp', 'logprob': -0.0...","[{'token': 'comp', 'logprob': -0.0184168349951...",comp.graphics,False,None
3,writes look collectively power throw bums dont...,talk.politics.misc,"{'content': [{'token': 'talk', 'logprob': -0.0...","[{'token': 'talk', 'logprob': -0.0023419829085...",sci.crypt,False,None
4,anyone idea get japanese league stats regularl...,rec.sport.baseball,"{'content': [{'token': 'rec', 'logprob': -0.00...","[{'token': 'rec', 'logprob': -0.00597215956076...",rec.sport.baseball,False,None
...,...,...,...,...,...,...,...
731,david sternlight writes article karl barrus wr...,sci.crypt,"{'content': [{'token': 'sci', 'logprob': -0.17...","[{'token': 'sci', 'logprob': -0.17696242034435...",sci.crypt,False,None
732,hello im looking information alphanumeric page...,sci.electronics,"{'content': [{'token': 'sci', 'logprob': -0.02...","[{'token': 'sci', 'logprob': -0.02991333045065...",sci.electronics,False,None
733,john r daker writes would like offocially nomi...,rec.autos,"{'content': [{'token': 'rec', 'logprob': -0.08...","[{'token': 'rec', 'logprob': -0.08121661096811...",rec.motorcycles,False,None
734,article writes looking information concerning ...,sci.space,"{'content': [{'token': 'sci', 'logprob': -0.00...","[{'token': 'sci', 'logprob': -0.00035386974923...",sci.space,False,None
